In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import matplotlib.pyplot as plt
import cv2
import os
import PIL
import pathlib
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [4]:
data_url ='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
data_dir = tf.keras.utils.get_file('flower_photos',origin = data_url, cache_dir='.',untar=True)


In [5]:
data_dir 

In [6]:
os.listdir(data_dir)

In [7]:
data_dir = pathlib.Path(data_dir)
data_dir

In [8]:
len(list(data_dir.glob('*/*.jpg')))

In [9]:
roses = list(data_dir.glob('roses/*.jpg'))
roses[:5]

In [10]:
PIL.Image.open(roses[0])

In [11]:
flower_images_dic = {
    'roses': list(data_dir.glob('roses/*.jpg')),
    'daisy': list(data_dir.glob('daisy/*.jpg')),
    'dandelion': list(data_dir.glob('dandelion/*.jpg')),
    'sunflowers': list(data_dir.glob('sunflowers/*.jpg')),
    'tulips': list(data_dir.glob('tulips/*.jpg'))                                
}

In [16]:
flower_labels_dic = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4
}

In [17]:
img = cv2.imread(str(flower_images_dic['roses'][0]))
img

In [18]:
X,y =[], []
for flower_name, images in flower_images_dic.items():
    for image in images:
        image = str(image)
        img = cv2.imread(image)
        resized_img = cv2.resize(img,(180,180))
        X.append(resized_img)
        y.append(flower_labels_dic[flower_name])

In [19]:
plt.figure(figsize=(15,20))
for i in range(16):
    plt.subplot(4,4, i + 1)
    plt.axis('off')
    plt.imshow(X[i])
    plt.title(y[i])
plt.show()
    

In [20]:
X = np.array(X)
y = np.array(y)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 0)

In [22]:
print("X_train Length: ", len(X_train))
print("X_test Length: ", len(X_test))

In [23]:
X_train_scaled = X_train / 255
X_test_scaled = X_test /255

In [24]:
model = Sequential([
                    layers.Conv2D(16, 3, padding='same', activation='relu'),
                    layers.MaxPooling2D(),
                    layers.Conv2D(32, 3, padding='same', activation='relu'),
                    layers.MaxPooling2D(),
                    layers.Conv2D(64, 3, padding='same', activation='relu'),
                    layers.MaxPooling2D(),
                    layers.Flatten(),
                    layers.Dense(128, activation='relu'),
                    layers.Dense(5, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [25]:
 history = model.fit(X_train_scaled, y_train, epochs=5, batch_size=64, verbose=1)

In [27]:
model.evaluate(X_test_scaled, y_test)

In [28]:
prediction = model.predict(X_test_scaled)

In [35]:
pred = np.argmax(prediction[1])
pred

In [36]:
y_test[1]

### Data Augmentation

In [38]:
data_augmentation = keras.Sequential([
                                      layers.experimental.preprocessing.RandomFlip('horizontal'),
                                      layers.experimental.preprocessing.RandomRotation(0.1),
                                      layers.experimental.preprocessing.RandomZoom(0.1)
])

In [39]:
model = Sequential([
                    data_augmentation,
                    layers.Conv2D(16, 3, padding='same', activation='relu'),
                    layers.MaxPooling2D(),
                    layers.Conv2D(32, 3, padding='same', activation='relu'),
                    layers.MaxPooling2D(),
                    layers.Conv2D(64, 3, padding='same', activation='relu'),
                    layers.MaxPooling2D(),
                    layers.Dropout(0.2),
                    layers.Flatten(),
                    layers.Dense(128, activation='relu'),
                    layers.Dense(5, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=64, verbose=1)

In [40]:
model.evaluate(X_test_scaled,y_test)

In [42]:
prediction = model.predict(X_test_scaled)

In [43]:
pred = []
for p in prediction:
    p = np.argmax(p)
    pred.append(p)

In [44]:
pred = np.array(pred)

In [46]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns 

In [49]:
print("Classification Report: \n", classification_report(y_test, pred))
print("-"*50)

print("Accuracy Report: \n", classification_report(y_test, pred))
print("-"*50)

plt.figure(figsize = (10,10))
plt.axis('off')
sns.heatmap(confusion_matrix(y_test,pred),annot = True, fmt='g');